In [1]:
from transformers import T5Config, T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments, TrainerCallback
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from Levenshtein import distance as levenshtein_distance
import json
import os
import torch
import pandas as pd

In [5]:
train_file_path = 'D:\\XJTLU\\YEAR4\\FYP\\TrainDirect.csv'
data = pd.read_csv(train_file_path)

train_val_data, test_data = train_test_split(data, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_val_data, test_size=0.1111, random_state=42)

device = torch.device("cuda")

In [6]:
class NLtoDSLDataSet(Dataset):
    def __init__(self, tokenizer, data, max_length=128):
        self.tokenizer = tokenizer
        self.data = data
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        input_text = item['input']
        target_text = item['output']
        
        input_encoding = self.tokenizer(input_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        target_encoding = self.tokenizer(target_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        
        inputs = {
            'input_ids': input_encoding['input_ids'].flatten(),
            'attention_mask': input_encoding['attention_mask'].flatten(),
            'labels': target_encoding['input_ids'].flatten()
        }
        
        return inputs

In [7]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

train_dataset = NLtoDSLDataSet(tokenizer, train_data)
val_dataset = NLtoDSLDataSet(tokenizer, val_data)

In [8]:
class EvalCallback(TrainerCallback):
    def on_train_end(self, args, state, control, **kwargs):
        print("Final evaluation on validation set.")
        metrics = trainer.evaluate()
        print(metrics)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    do_train=True,
    evaluation_strategy="no",
    load_best_model_at_end=True,
    logging_steps=10,
    save_strategy="no"
) 

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EvalCallback()]
)

trainer.train()

d:\Anaconda3\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


  0%|          | 0/750 [00:00<?, ?it/s]

{'loss': 9.2717, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.04}
{'loss': 9.127, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.08}
{'loss': 8.6384, 'learning_rate': 3e-06, 'epoch': 0.12}
{'loss': 8.8759, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.16}
{'loss': 8.4959, 'learning_rate': 5e-06, 'epoch': 0.2}
{'loss': 7.8521, 'learning_rate': 6e-06, 'epoch': 0.24}
{'loss': 7.3523, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.28}
{'loss': 7.1268, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.32}
{'loss': 6.5314, 'learning_rate': 9e-06, 'epoch': 0.36}
{'loss': 6.0294, 'learning_rate': 1e-05, 'epoch': 0.4}
{'loss': 5.2557, 'learning_rate': 1.1000000000000001e-05, 'epoch': 0.44}
{'loss': 4.7159, 'learning_rate': 1.2e-05, 'epoch': 0.48}
{'loss': 4.1254, 'learning_rate': 1.3000000000000001e-05, 'epoch': 0.52}
{'loss': 3.4779, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.56}
{'loss': 3.2558, 'learning_rate': 1.5e-05, 'epoch': 0.6}
{'loss': 2.688, 'learning_r

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.0013685429003089666, 'eval_runtime': 12.7427, 'eval_samples_per_second': 39.238, 'eval_steps_per_second': 4.944, 'epoch': 3.0}
{'eval_loss': 0.0013685429003089666, 'eval_runtime': 12.7427, 'eval_samples_per_second': 39.238, 'eval_steps_per_second': 4.944, 'epoch': 3.0}


TrainOutput(global_step=750, training_loss=1.5939265880187352, metrics={'train_runtime': 698.7311, 'train_samples_per_second': 17.174, 'train_steps_per_second': 1.073, 'train_loss': 1.5939265880187352, 'epoch': 3.0})

In [9]:
evaluation_result = trainer.evaluate()
print(evaluation_result)

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.0013685429003089666, 'eval_runtime': 3.2448, 'eval_samples_per_second': 154.095, 'eval_steps_per_second': 19.416, 'epoch': 3.0}


In [10]:
def generate_prediction(input_text, model, tokenizer, device):
    model.eval()
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    output_ids = model.generate(input_ids)[0]
    output_text = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output_text

sample_input = "How about we assign 22 percentage to fixed-income securities and 87% to small-cap stocks in ValueInvest?"
print(generate_prediction(sample_input, model, tokenizer, device))

d:\Anaconda3\lib\site-packages\transformers\generation\utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Strategy for ValueInvest: 1. Set 22% of assets to fixed-income securities. 2. Set


In [11]:
def evaluate_model(test_data, model, tokenizer, device, generate_func):
    rouge = Rouge()
    total_bleu_score = 0
    total_rouge_score = {"rouge-1": {"f": 0, "p": 0, "r": 0}, "rouge-2": {"f": 0, "p": 0, "r": 0}, "rouge-l": {"f": 0, "p": 0, "r": 0}}
    total_levenshtein = 0
    correct = 0
    total = 0

    for index, row in test_data.iterrows():
        input_text = row['input']
        expected_output = row['output']
        predicted_output = generate_func(input_text, model, tokenizer, device)
        
        # BLEU Score
        reference = [expected_output.lower().split()]
        candidate = predicted_output.lower().split()
        bleu_score = sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25))
        total_bleu_score += bleu_score
        
        # ROUGE Score
        scores = rouge.get_scores(predicted_output, expected_output)
        for key in total_rouge_score:
            for metric in total_rouge_score[key]:
                total_rouge_score[key][metric] += scores[0][key][metric]
        
        # Levenshtein Distance
        lev_dist = levenshtein_distance(predicted_output.lower(), expected_output.lower())
        total_levenshtein += lev_dist
        
        # Accuracy
        if predicted_output.strip().lower() == expected_output.strip().lower():
            correct += 1

        total += 1

    average_bleu = total_bleu_score / total
    average_rouge = {key: {k: v / total for k, v in total_rouge_score[key].items()} for key in total_rouge_score}
    average_levenshtein = total_levenshtein / total
    accuracy = correct / total
    
    return {
        "average_bleu": average_bleu,
        "average_rouge": average_rouge,
        "average_levenshtein": average_levenshtein,
        "accuracy": accuracy
    }

def print_evaluation_results(title, results):
    print(title + ":")
    for key, value in results.items():
        if isinstance(value, dict):
            print(f"  {key}: {{")
            for subkey, subvalue in value.items():
                print(f"    {subkey}: {subvalue}")
            print("  }")
        else:
            print(f"  {key}: {value}")
    print()

In [12]:
results = evaluate_model(test_data, model, tokenizer, device, generate_prediction)
print_evaluation_results("Evaluation Results", results)

Evaluation Results:
  average_bleu: 0.04667631108253263
  average_rouge: {
    rouge-1: {'f': 0.485942681772611, 'p': 0.9825512820512808, 'r': 0.32335339954163456}
    rouge-2: {'f': 0.3889074295274914, 'p': 0.9785271950271963, 'r': 0.24347270615563268}
    rouge-l: {'f': 0.485942681772611, 'p': 0.9825512820512808, 'r': 0.32335339954163456}
  }
  average_levenshtein: 247.66
  accuracy: 0.0

